In [1]:
import cv2

In [2]:
from tensorflow.keras.models import load_model

In [3]:
import numpy as np

In [4]:
import os
print(os.listdir())

['.ipynb_checkpoints', 'emotion_detection_santxa.ipynb', 'emotion_sets.hdf5']


In [5]:
model = load_model("emotion_sets.hdf5",compile=False)

In [6]:
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Normal']

In [7]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [8]:
cap = cv2.VideoCapture(0)

In [9]:
while True:
    # Capture frame from webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

        # Crop and preprocess face
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        # Only proceed if face is properly detected
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = np.expand_dims(roi, axis=0)
            roi = np.expand_dims(roi, axis=-1)

            # Predict emotion
            prediction = model.predict(roi, verbose=0)[0]
            label = emotion_labels[np.argmax(prediction)]

            # Display emotion label above the face
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Show the live video
    cv2.imshow('Emotion Detector', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

ValueError: Input 0 of layer "model_1" is incompatible with the layer: expected shape=(None, 64, 64, 1), found shape=(1, 48, 48)

In [ ]:
cv2.release()
cv2.destroyAllWindows()